In [1]:
from pylab import plt
import pandas as pd
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import datetime as dt
import scipy as sc
from pylab import mpl
import scipy.integrate as sci
import scipy.stats as scs
import pandas_datareader as web
import statsmodels.api as sm
import enum
import scipy.optimize as spo
from numpy.fft import *
from yahoo_fin import options
from dawp import *
from mystic.solvers import fmin, fmin_powell, diffev, diffev2, sparsity,lattice
import math
import sys
import yfinance as yf

sys.path.append('../')
sys.path.append('C:/Users/kiedy/dx')

import autograd.numpy as agnp
from autograd import grad

np.set_printoptions(suppress=True, formatter={'all': lambda x: '%5.4f' % x})

import datetime as dt
import simpleplotly as spt
from fftoptionlib import *
import plotly
import plotly.offline as py
from scipy.stats import norm

In [2]:
options1=pd.DataFrame([43.10, 35.60, 22.90, 20.20, 13.30],
                     columns=['price'])
options1['strike']=(1090,1100,1120,1125,1140)
options1['maturity']=dt.datetime(2002,5,17)
options1['T_2']=0.088


options2=pd.DataFrame([84.50, 64.30, 39.50, 33.50, 30.70, 28.00,  25.60, 23.20,19.10,15.30,12.10,10.90],
                     columns=['price'])
options2['strike']=(1050,1075,1110,1120,1125,1130,1135,1140,1150,1160,1170,1175)
options2['maturity']=dt.datetime(2002,6,21)
options2['T_2']=0.184


options3=pd.DataFrame([161.60, 144.80, 120.10, 100.70, 82.50,65.50,51.00,45.50,38.10,27.70,19.60,13.20],
                     columns=['price'])
options3['strike']=(975,995,1025,1050,1075,1100,1125,1135,1150,1175,1200,1225)
options3['maturity']=dt.datetime(2002,9,20)
options3['T_2']=0.436



options4=pd.DataFrame([173.30,157.00,133.10,114.80,97.60,81.20,66.90,58.90,53.90,42.50,33.00,24.90,18.30,13.20],
                     columns=['price'])
options4['strike']=(975,995,1025,1050,1075,1100,1125,1140,1150,1175,1200,1225,1250,1275)
options4['maturity']=dt.datetime(2002,12,20)
options4['T_2']=0.692


options5=pd.DataFrame([146.50,96.20,81.70,68.30,56.60,46.10,36.90,29.30,22.50,17.20,12.80],
                     columns=['price'])
options5['strike']=(1025,1100,1125,1150,1175,1200,1225,1250,1275,1300,1325)
options5['maturity']=dt.datetime(2003,3,21)
options5['T_2']=0.936


options6=pd.DataFrame([182.10,143.00,111.30,97.00,83.30,60.90,49.80,41.20,27.10,17.10,10.10],
                     columns=['price'])
options6['strike']=(995,1050,1100,1125,1150,1200,1225,1250,1300,1350,1400)
options6['maturity']=dt.datetime(2003,6,20)
options6['T_2']=1.192

options7=pd.DataFrame([171.40,140.40,112.80,99.80,66.90,49.50,35.70,25.20,17.00,12.20],
                     columns=['price'])
options7['strike']=(1050,1100,1150,1175,1250,1300,1350,1400,1450,1500)
options7['maturity']=dt.datetime(2003,12,19)
options7['T_2']=1.708

frames=[options1, options2, options3, options4, options5, options6, options7]

options=pd.concat(frames, ignore_index=True)
options['pricing_date'] = dt.datetime(2002, 4, 18)
options['T']=(options['maturity']-options['pricing_date']).dt.days/365.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


r=0.019
q=0.012
S0=1124.47
i = np.complex(0.0, 1.0)

# Metoda COS

In [3]:
def cf(u, T, r,q, sigma, lamb, mu, delta):
    ''' Funkcja charakterystyczna w modelu Mertona '''
    b = r - q - 0.5 * sigma ** 2 - lamb * (np.exp(mu+delta**2/2)-1)
    value = np.exp(T*(1j * b * u - 0.5 * u ** 2 * sigma ** 2 + lamb*(np.exp(1j*mu*u-sigma**2 * u**2 / 2)-1)))
    return value

In [4]:
N=2**12
L=10
    
def CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K, sigma,lamb,mu,delta,N,L,CALL):


    # cf   - Characteristic function is a function, in the book denoted by \varphi
    # CP   - C for call and P for put
    # S0   - Initial stock price
    # r    - Interest rate (constant)
    # T    - Time to maturity
    # K    - List of strikes
    # N    - Number of expansion terms
    # L    - Size of truncation domain (typ.:L=8 or L=10)
        
    # Reshape K to become a column vector

    if K is not np.array:
        K = np.array(K).reshape([len(K),1])
        
    x0 = np.log(S0 / K)
    
    # cumulants
    c1=T*(r-q-sigma**2/2-lamb*(np.exp(mu+delta**2/2)-1)+lamb*mu)
    c2=T*(sigma**2+ lamb*mu**2+lamb*delta**2)  
    c4=T*lamb*(mu**4 +6*mu**2 * delta**2 +3* delta **4)

    a=c1+x0-L*np.sqrt(c2+np.sqrt(c4))
    b=c1+x0+L*np.sqrt(c2+np.sqrt(c4))
    # Summation from k = 0 to k=N-1

    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);  

    # Determine coefficients for put prices  

    H_k = CallPutCoefficients(a,b,k)   
    mat = np.exp(1j * np.outer((x0 - a) , u))
    temp = cf(u, T, r,q, sigma, lamb, mu, delta) * H_k 
    temp[0] = 0.5 * temp[0]    
    value = np.exp(-r * T) * K * np.real(mat.dot(temp))
    if CALL == True:
        return  value + np.exp(-q*T)*S0 - K*np.exp(-r*T)
    else:
        return value

# Determine coefficients for put prices 

def CallPutCoefficients(a,b,k):
    
    c = a
    d = 0.0
    coef = Chi_Psi(a,b,c,d,k)
    Chi_k = coef["chi"]
    Psi_k = coef["psi"]
    if a<b and b<0.0:
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k) 
    elif a>0 and a<b:
        H_k = np.zeros([len(k),1])
    else:
        H_k = 2.0 / (b - a) * (- Chi_k + Psi_k)
    
    return H_k    

def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value

In [5]:
def Merton_error_function(p0):

    global m, local_opt, opt_brute
    m=1
    sigma, lamb,mu, delta=p0
    if sigma<0.0 or lamb<0 or delta<0.0  or mu>0.5 or mu<-0.5:
        return 100
    se = []
    for row, option in options.iterrows():
        model_value =CallPutOptionPriceCOSMthd(cf,S0,r,q,option['T_2'],(option['strike'],), sigma,lamb,mu,delta,N,L,True)[0][0]
        se.append((model_value - option['price']) ** 2)
    RMSE = (sum(se) / len(se))**(0.5)
    if m % 1 == 0:
        print('%4d |' % m, np.array(p0), '| %7.3f' % RMSE)
        
    m += 1
    return RMSE

Merton_error_function((0.1806,0,0.000010350,0.000014239))
Merton_error_function((0.128, 0.187, -0.400, 0.001))

   1 | [0.1806 0.0000 0.0000 0.0000] |   6.738
   1 | [0.1280 0.1870 -0.4000 0.0010] |   2.816


2.8163614873006644

In [17]:
def Merton_calibration_full():
    i=0
    global opt_brute, local_opt, opt_fmin
    opt_brute = spo.brute(Merton_error_function, ((0.05,0.201, 0.1),(0.1,0.5, 0.05), (-0.4,0,0.05), (0.01, 1.51, 0.1)), finish=None)
    opt_fmin = spo.fmin(Merton_error_function, opt_brute, xtol=0.000001, ftol=0.000001, maxiter=750, maxfun=900)
    return opt_fmin

Merton_calibration_full()

   1 | [0.0500 0.1000 -0.4000 0.0100] |  27.858
   1 | [0.0500 0.1000 -0.4000 0.1100] |  27.658
   1 | [0.0500 0.1000 -0.4000 0.2100] |  27.122
   1 | [0.0500 0.1000 -0.4000 0.3100] |  26.251
   1 | [0.0500 0.1000 -0.4000 0.4100] |  25.050
   1 | [0.0500 0.1000 -0.4000 0.5100] |  23.544
   1 | [0.0500 0.1000 -0.4000 0.6100] |  21.816
   1 | [0.0500 0.1000 -0.4000 0.7100] |  20.069
   1 | [0.0500 0.1000 -0.4000 0.8100] |  18.773
   1 | [0.0500 0.1000 -0.4000 0.9100] |  18.811
   1 | [0.0500 0.1000 -0.4000 1.0100] |  21.332
   1 | [0.0500 0.1000 -0.4000 1.1100] |  27.109
   1 | [0.0500 0.1000 -0.4000 1.2100] |  36.367
   1 | [0.0500 0.1000 -0.4000 1.3100] |  49.268
   1 | [0.0500 0.1000 -0.4000 1.4100] |  66.187
   1 | [0.0500 0.1000 -0.3500 0.0100] |  29.359
   1 | [0.0500 0.1000 -0.3500 0.1100] |  29.147
   1 | [0.0500 0.1000 -0.3500 0.2100] |  28.581
   1 | [0.0500 0.1000 -0.3500 0.3100] |  27.660
   1 | [0.0500 0.1000 -0.3500 0.4100] |  26.391
   1 | [0.0500 0.1000 -0.3500 0.5100] | 

   1 | [0.0500 0.1500 -0.2500 0.7100] |  19.628
   1 | [0.0500 0.1500 -0.2500 0.8100] |  22.141
   1 | [0.0500 0.1500 -0.2500 0.9100] |  28.910
   1 | [0.0500 0.1500 -0.2500 1.0100] |  39.999
   1 | [0.0500 0.1500 -0.2500 1.1100] |  55.340
   1 | [0.0500 0.1500 -0.2500 1.2100] |  75.155
   1 | [0.0500 0.1500 -0.2500 1.3100] |  99.856
   1 | [0.0500 0.1500 -0.2500 1.4100] | 129.951
   1 | [0.0500 0.1500 -0.2000 0.0100] |  31.860
   1 | [0.0500 0.1500 -0.2000 0.1100] |  31.483
   1 | [0.0500 0.1500 -0.2000 0.2100] |  30.483
   1 | [0.0500 0.1500 -0.2000 0.3100] |  28.884
   1 | [0.0500 0.1500 -0.2000 0.4100] |  26.764
   1 | [0.0500 0.1500 -0.2000 0.5100] |  24.339
   1 | [0.0500 0.1500 -0.2000 0.6100] |  22.165
   1 | [0.0500 0.1500 -0.2000 0.7100] |  21.463
   1 | [0.0500 0.1500 -0.2000 0.8100] |  24.047
   1 | [0.0500 0.1500 -0.2000 0.9100] |  31.149
   1 | [0.0500 0.1500 -0.2000 1.0100] |  42.872
   1 | [0.0500 0.1500 -0.2000 1.1100] |  59.112
   1 | [0.0500 0.1500 -0.2000 1.2100] | 

   1 | [0.0500 0.2000 -0.1000 1.4100] | 207.103
   1 | [0.0500 0.2000 -0.0500 0.0100] |  37.348
   1 | [0.0500 0.2000 -0.0500 0.1100] |  36.753
   1 | [0.0500 0.2000 -0.0500 0.2100] |  35.186
   1 | [0.0500 0.2000 -0.0500 0.3100] |  32.731
   1 | [0.0500 0.2000 -0.0500 0.4100] |  29.643
   1 | [0.0500 0.2000 -0.0500 0.5100] |  26.646
   1 | [0.0500 0.2000 -0.0500 0.6100] |  25.474
   1 | [0.0500 0.2000 -0.0500 0.7100] |  28.873
   1 | [0.0500 0.2000 -0.0500 0.8100] |  38.543
   1 | [0.0500 0.2000 -0.0500 0.9100] |  54.265
   1 | [0.0500 0.2000 -0.0500 1.0100] |  75.545
   1 | [0.0500 0.2000 -0.0500 1.1100] | 102.279
   1 | [0.0500 0.2000 -0.0500 1.2100] | 134.659
   1 | [0.0500 0.2000 -0.0500 1.3100] | 172.995
   1 | [0.0500 0.2000 -0.0500 1.4100] | 217.598
   1 | [0.0500 0.2500 -0.4000 0.0100] |  12.646
   1 | [0.0500 0.2500 -0.4000 0.1100] |  12.184
   1 | [0.0500 0.2500 -0.4000 0.2100] |  10.991
   1 | [0.0500 0.2500 -0.4000 0.3100] |   9.263
   1 | [0.0500 0.2500 -0.4000 0.4100] | 

   1 | [0.0500 0.3000 -0.3000 0.6100] |  19.023
   1 | [0.0500 0.3000 -0.3000 0.7100] |  30.903
   1 | [0.0500 0.3000 -0.3000 0.8100] |  46.702
   1 | [0.0500 0.3000 -0.3000 0.9100] |  66.906
   1 | [0.0500 0.3000 -0.3000 1.0100] |  92.184
   1 | [0.0500 0.3000 -0.3000 1.1100] | 123.091
   1 | [0.0500 0.3000 -0.3000 1.2100] | 159.968
   1 | [0.0500 0.3000 -0.3000 1.3100] | 203.064
   1 | [0.0500 0.3000 -0.3000 1.4100] | 252.526
   1 | [0.0500 0.3000 -0.2500 0.0100] |  20.233
   1 | [0.0500 0.3000 -0.2500 0.1100] |  19.520
   1 | [0.0500 0.3000 -0.2500 0.2100] |  17.671
   1 | [0.0500 0.3000 -0.2500 0.3100] |  14.960
   1 | [0.0500 0.3000 -0.2500 0.4100] |  12.488
   1 | [0.0500 0.3000 -0.2500 0.5100] |  13.408
   1 | [0.0500 0.3000 -0.2500 0.6100] |  20.371
   1 | [0.0500 0.3000 -0.2500 0.7100] |  32.528
   1 | [0.0500 0.3000 -0.2500 0.8100] |  49.246
   1 | [0.0500 0.3000 -0.2500 0.9100] |  70.765
   1 | [0.0500 0.3000 -0.2500 1.0100] |  97.585
   1 | [0.0500 0.3000 -0.2500 1.1100] | 

   1 | [0.0500 0.3500 -0.1500 1.3100] | 271.384
   1 | [0.0500 0.3500 -0.1500 1.4100] | 330.243
   1 | [0.0500 0.3500 -0.1000 0.0100] |  33.004
   1 | [0.0500 0.3500 -0.1000 0.1100] |  32.025
   1 | [0.0500 0.3500 -0.1000 0.2100] |  29.506
   1 | [0.0500 0.3500 -0.1000 0.3100] |  25.865
   1 | [0.0500 0.3500 -0.1000 0.4100] |  22.552
   1 | [0.0500 0.3500 -0.1000 0.5100] |  23.337
   1 | [0.0500 0.3500 -0.1000 0.6100] |  32.216
   1 | [0.0500 0.3500 -0.1000 0.7100] |  49.056
   1 | [0.0500 0.3500 -0.1000 0.8100] |  72.636
   1 | [0.0500 0.3500 -0.1000 0.9100] | 102.486
   1 | [0.0500 0.3500 -0.1000 1.0100] | 138.520
   1 | [0.0500 0.3500 -0.1000 1.1100] | 180.775
   1 | [0.0500 0.3500 -0.1000 1.2100] | 229.261
   1 | [0.0500 0.3500 -0.1000 1.3100] | 283.847
   1 | [0.0500 0.3500 -0.1000 1.4100] | 344.144
   1 | [0.0500 0.3500 -0.0500 0.0100] |  36.306
   1 | [0.0500 0.3500 -0.0500 0.1100] |  35.269
   1 | [0.0500 0.3500 -0.0500 0.2100] |  32.602
   1 | [0.0500 0.3500 -0.0500 0.3100] | 

...

   1 | [0.1500 0.3000 -0.0500 0.2100] |   7.043
   1 | [0.1500 0.3000 -0.0500 0.3100] |   8.200
   1 | [0.1500 0.3000 -0.0500 0.4100] |  13.245
   1 | [0.1500 0.3000 -0.0500 0.5100] |  21.906
   1 | [0.1500 0.3000 -0.0500 0.6100] |  33.983
   1 | [0.1500 0.3000 -0.0500 0.7100] |  49.800
   1 | [0.1500 0.3000 -0.0500 0.8100] |  69.938
   1 | [0.1500 0.3000 -0.0500 0.9100] |  95.097
   1 | [0.1500 0.3000 -0.0500 1.0100] | 125.983
   1 | [0.1500 0.3000 -0.0500 1.1100] | 163.183
   1 | [0.1500 0.3000 -0.0500 1.2100] | 207.039
   1 | [0.1500 0.3000 -0.0500 1.3100] | 257.578
   1 | [0.1500 0.3000 -0.0500 1.4100] | 314.500
   1 | [0.1500 0.3500 -0.4000 0.0100] |  21.668
   1 | [0.1500 0.3500 -0.4000 0.1100] |  22.257
   1 | [0.1500 0.3500 -0.4000 0.2100] |  23.887
   1 | [0.1500 0.3500 -0.4000 0.3100] |  26.717
   1 | [0.1500 0.3500 -0.4000 0.4100] |  30.977
   1 | [0.1500 0.3500 -0.4000 0.5100] |  36.969
   1 | [0.1500 0.3500 -0.4000 0.6100] |  45.068
   1 | [0.1500 0.3500 -0.4000 0.7100] | 

   1 | [0.1500 0.4000 -0.3000 0.8100] |  80.491
   1 | [0.1500 0.4000 -0.3000 0.9100] | 104.603
   1 | [0.1500 0.4000 -0.3000 1.0100] | 134.728
   1 | [0.1500 0.4000 -0.3000 1.1100] | 171.627
   1 | [0.1500 0.4000 -0.3000 1.2100] | 215.717
   1 | [0.1500 0.4000 -0.3000 1.3100] | 266.929
   1 | [0.1500 0.4000 -0.3000 1.4100] | 324.738
   1 | [0.1500 0.4000 -0.2500 0.0100] |   9.268
   1 | [0.1500 0.4000 -0.2500 0.1100] |  10.083
   1 | [0.1500 0.4000 -0.2500 0.2100] |  12.463
   1 | [0.1500 0.4000 -0.2500 0.3100] |  16.778
   1 | [0.1500 0.4000 -0.2500 0.4100] |  23.377
   1 | [0.1500 0.4000 -0.2500 0.5100] |  32.630
   1 | [0.1500 0.4000 -0.2500 0.6100] |  45.021
   1 | [0.1500 0.4000 -0.2500 0.7100] |  61.179
   1 | [0.1500 0.4000 -0.2500 0.8100] |  81.871
   1 | [0.1500 0.4000 -0.2500 0.9100] | 107.956
   1 | [0.1500 0.4000 -0.2500 1.0100] | 140.265
   1 | [0.1500 0.4000 -0.2500 1.1100] | 179.432
   1 | [0.1500 0.4000 -0.2500 1.2100] | 225.707
   1 | [0.1500 0.4000 -0.2500 1.3100] | 

   1 | [0.1500 0.4500 -0.1000 0.0100] |   5.364
   1 | [0.1500 0.4500 -0.1000 0.1100] |   5.132
   1 | [0.1500 0.4500 -0.1000 0.2100] |   6.413
   1 | [0.1500 0.4500 -0.1000 0.3100] |  11.724
   1 | [0.1500 0.4500 -0.1000 0.4100] |  20.989
   1 | [0.1500 0.4500 -0.1000 0.5100] |  34.203
   1 | [0.1500 0.4500 -0.1000 0.6100] |  51.872
   1 | [0.1500 0.4500 -0.1000 0.7100] |  74.732
   1 | [0.1500 0.4500 -0.1000 0.8100] | 103.582
   1 | [0.1500 0.4500 -0.1000 0.9100] | 139.115
   1 | [0.1500 0.4500 -0.1000 1.0100] | 181.726
   1 | [0.1500 0.4500 -0.1000 1.1100] | 231.386
   1 | [0.1500 0.4500 -0.1000 1.2100] | 287.636
   1 | [0.1500 0.4500 -0.1000 1.3100] | 349.675
   1 | [0.1500 0.4500 -0.1000 1.4100] | 416.393
   1 | [0.1500 0.4500 -0.0500 0.0100] |   6.718
   1 | [0.1500 0.4500 -0.0500 0.1100] |   6.218
   1 | [0.1500 0.4500 -0.0500 0.2100] |   6.751
   1 | [0.1500 0.4500 -0.0500 0.3100] |  11.784
   1 | [0.1500 0.4500 -0.0500 0.4100] |  21.466
   1 | [0.1500 0.4500 -0.0500 0.5100] | 

   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1303 0.1369 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1303 0.1369 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1368 -0.5000 0.0086] |   2.615
   1 | [0.1304 0.1369 -0.5000 0.0086] | 

array([0.1304, 0.1368, -0.5000, 0.0086])

NameError: name 'opt_fmin' is not defined

# dla wielu K

In [6]:
N=2**12
L=10
    
def CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma, lamb,mu,delta,N,L,CALL):


    # cf   - Characteristic function is a function, in the book denoted by \varphi
    # CP   - C for call and P for put
    # S0   - Initial stock price
    # r    - Interest rate (constant)
    # T    - Time to maturity
    # K    - List of strikes
    # N    - Number of expansion terms
    # L    - Size of truncation domain (typ.:L=8 or L=10)
        
    # Reshape K to become a column vector

    if K is not np.array:
        K = np.array(K).reshape([len(K),1])
        
    x0 = np.log(S0 / K)
    
    # kumulanty
    c1=T*(r-q-sigma**2/2-lamb*(np.exp(mu+delta**2/2)-1)+lamb*mu)
    c2=T*(sigma**2+ lamb*mu**2+lamb*delta**2)  
    c4=T*lamb*(mu**4 +6*mu**2 * delta**2 +3* delta **4)
    
    # kumulanty z automatycznego różniczkowania
#     c1=grad_g(0.)
#     c2=grad_g2(0.)
#     c4=grad_g4(0.)

    # kumulanty z metody różnic skończonych
#     c1=(G(h)-G(-h))/(2*h)
#     c2=(G(h)-2*G(0)+G(-h))/(h**2)
#     c4=(G(3*h)-2*G(2*h)+4*G(0)-G(h)-G(-h)-2*G(-2*h)+G(-3*h))/(4*h**4)

    a=c1-L*np.sqrt(c2+np.sqrt(c4))
    b=c1+L*np.sqrt(c2+np.sqrt(c4))
    # Summation from k = 0 to k=N-1

    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);  

    # Determine coefficients for put prices  

    H_k = CallPutCoefficients(a,b,k)   
    mat = np.exp(1j * np.outer((x0 - a) , u))
    temp = cf(u, T, r,q, sigma, lamb, mu, delta) * H_k 
    temp[0] = 0.5 * temp[0]    
    value = np.exp(-r * T) * K * np.real(mat.dot(temp))
    if CALL == True:
        return  value + np.exp(-q*T)*S0 - K*np.exp(-r*T)
    else:
        return value

# Determine coefficients for put prices 

def CallPutCoefficients(a,b,k):
    
    c = a
    d = 0.0
    coef = Chi_Psi(a,b,c,d,k)
    Chi_k = coef["chi"]
    Psi_k = coef["psi"]
    if a<b and b<0.0:
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k) 
    elif a>0 and a<b:
        H_k = np.zeros([len(k),1])
    else:
        H_k = 2.0 / (b - a) * (- Chi_k + Psi_k)
    
    return H_k    

def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value

In [7]:
m=0
def Merton_error_function(p0):
    
    global m
    sigma,lamb, mu, delta = p0
    if sigma<0.0 or lamb<0 or delta<=0.0 or delta>0.8 or mu>0.5 or mu<-0.4:
        return 100
#     if sigma<=0:
#         return 100
    se1 = []
    K = np.array(options1['strike'])
    T=options['T_2'][0]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma, lamb, mu, delta,N,L,True)
    i=0
    for row, option in options1.iterrows():
        se1.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se2 = []
    K = np.array(options2['strike'])
    T=options['T_2'][10]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma,lamb, mu, delta,N,L,True)
    i=0
    for row, option in options2.iterrows():
        se2.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se3 = []    
    K = np.array(options3['strike'])
    T=options['T_2'][18]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma,lamb, mu, delta,N,L,True)
    i=0
    for row, option in options3.iterrows():
        se3.append((model_value[i] - option['price']) ** 2)
        i+=1        
        
        
    se4 = []    
    K = np.array(options4['strike'])
    T=options['T_2'][37]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma,lamb, mu, delta,N,L,True)
    i=0
    for row, option in options4.iterrows():
        se4.append((model_value[i] - option['price']) ** 2)
        i+=1        

    se5 = []    
    K = np.array(options5['strike'])
    T=options['T_2'][48]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma,lamb, mu, delta,N,L,True)
    i=0
    for row, option in options5.iterrows():
        se5.append((model_value[i] - option['price']) ** 2)
        i+=1 
        
    se6 = []    
    K = np.array(options6['strike'])
    T=options['T_2'][58]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma,lamb, mu, delta,N,L,True)
    i=0
    for row, option in options6.iterrows():
        se6.append((model_value[i] - option['price']) ** 2)
        i+=1
        
        
    se7 = []    
    K = np.array(options7['strike'])
    T=options['T_2'][69]
    model_value=CallPutOptionPriceCOSMthd(cf,S0,r,q,T,K,sigma,lamb, mu, delta,N,L,True)
    i=0
    for row, option in options7.iterrows():
        se7.append((model_value[i] - option['price']) ** 2)
        i+=1
        
    RMSE = ((sum(se1)+sum(se2)+sum(se3)+sum(se4)+sum(se5)+sum(se6)+sum(se7)) / (len(se1)+len(se2)+len(se3)+len(se4)+len(se5)+len(se6)+len(se7)))**0.5
    if np.isnan(RMSE):
        RMSE=100
    if m % 1 == 0:
        print('%4d |' % m, np.array(p0), '| %7.3f' % RMSE)
        
    m += 1
    return RMSE
Merton_error_function((0.1806,0.1535,0.000010350,0.000014239))

   0 | [0.1806 0.1535 0.0000 0.0000] |   7.031


array([7.0308])

In [9]:
opt_brute = spo.brute(Merton_error_function, ((0.05,0.201, 0.1),(0.01,2, 0.4), (-0.4,0,0.05), (0.01, 1, 0.1)), finish=None)

   1 | [0.0500 0.0100 -0.4000 0.0100] |  37.722
   2 | [0.0500 0.0100 -0.4000 0.1100] |  37.701
   3 | [0.0500 0.0100 -0.4000 0.2100] |  37.644
   4 | [0.0500 0.0100 -0.4000 0.3100] |  37.551
   5 | [0.0500 0.0100 -0.4000 0.4100] |  37.417
   6 | [0.0500 0.0100 -0.4000 0.5100] |  37.241
   7 | [0.0500 0.0100 -0.4000 0.6100] |  37.015
   8 | [0.0500 0.0100 -0.4000 0.7100] |  36.734
   9 | [0.0500 0.0100 -0.3500 0.0100] |  37.860
  10 | [0.0500 0.0100 -0.3500 0.1100] |  37.838
  11 | [0.0500 0.0100 -0.3500 0.2100] |  37.779
  12 | [0.0500 0.0100 -0.3500 0.3100] |  37.680
  13 | [0.0500 0.0100 -0.3500 0.4100] |  37.540
  14 | [0.0500 0.0100 -0.3500 0.5100] |  37.354
  15 | [0.0500 0.0100 -0.3500 0.6100] |  37.116
  16 | [0.0500 0.0100 -0.3500 0.7100] |  36.821
  17 | [0.0500 0.0100 -0.3000 0.0100] |  38.005
  18 | [0.0500 0.0100 -0.3000 0.1100] |  37.982
  19 | [0.0500 0.0100 -0.3000 0.2100] |  37.919
  20 | [0.0500 0.0100 -0.3000 0.3100] |  37.816
  21 | [0.0500 0.0100 -0.3000 0.4100] | 

 173 | [0.0500 0.8100 -0.1500 0.4100] |  29.336
 174 | [0.0500 0.8100 -0.1500 0.5100] |  54.505
 175 | [0.0500 0.8100 -0.1500 0.6100] |  88.239
 176 | [0.0500 0.8100 -0.1500 0.7100] | 130.727
 177 | [0.0500 0.8100 -0.1000 0.0100] |  26.041
 178 | [0.0500 0.8100 -0.1000 0.1100] |  23.814
 179 | [0.0500 0.8100 -0.1000 0.2100] |  18.789
 180 | [0.0500 0.8100 -0.1000 0.3100] |  17.242
 181 | [0.0500 0.8100 -0.1000 0.4100] |  30.605
 182 | [0.0500 0.8100 -0.1000 0.5100] |  56.755
 183 | [0.0500 0.8100 -0.1000 0.6100] |  92.816
 184 | [0.0500 0.8100 -0.1000 0.7100] | 137.816
 185 | [0.0500 0.8100 -0.0500 0.0100] |  33.241
 186 | [0.0500 0.8100 -0.0500 0.1100] |  30.878
 187 | [0.0500 0.8100 -0.0500 0.2100] |  25.425
 188 | [0.0500 0.8100 -0.0500 0.3100] |  22.189
 189 | [0.0500 0.8100 -0.0500 0.4100] |  33.532
 190 | [0.0500 0.8100 -0.0500 0.5100] |  60.399
 191 | [0.0500 0.8100 -0.0500 0.6100] |  98.395
 192 | [0.0500 0.8100 -0.0500 0.7100] | 145.554
 193 | [0.0500 1.2100 -0.4000 0.0100] | 

 344 | [0.1500 0.0100 -0.3000 0.7100] |  10.963
 345 | [0.1500 0.0100 -0.2500 0.0100] |  11.874
 346 | [0.1500 0.0100 -0.2500 0.1100] |  11.856
 347 | [0.1500 0.0100 -0.2500 0.2100] |  11.806
 348 | [0.1500 0.0100 -0.2500 0.3100] |  11.725
 349 | [0.1500 0.0100 -0.2500 0.4100] |  11.610
 350 | [0.1500 0.0100 -0.2500 0.5100] |  11.460
 351 | [0.1500 0.0100 -0.2500 0.6100] |  11.273
 352 | [0.1500 0.0100 -0.2500 0.7100] |  11.047
 353 | [0.1500 0.0100 -0.2000 0.0100] |  11.987
 354 | [0.1500 0.0100 -0.2000 0.1100] |  11.967
 355 | [0.1500 0.0100 -0.2000 0.2100] |  11.915
 356 | [0.1500 0.0100 -0.2000 0.3100] |  11.829
 357 | [0.1500 0.0100 -0.2000 0.4100] |  11.708
 358 | [0.1500 0.0100 -0.2000 0.5100] |  11.550
 359 | [0.1500 0.0100 -0.2000 0.6100] |  11.353
 360 | [0.1500 0.0100 -0.2000 0.7100] |  11.117
 361 | [0.1500 0.0100 -0.1500 0.0100] |  12.081
 362 | [0.1500 0.0100 -0.1500 0.1100] |  12.061
 363 | [0.1500 0.0100 -0.1500 0.2100] |  12.006
 364 | [0.1500 0.0100 -0.1500 0.3100] | 

 516 | [0.1500 1.2100 -0.4000 0.3100] | 103.892
 517 | [0.1500 1.2100 -0.4000 0.4100] | 116.616
 518 | [0.1500 1.2100 -0.4000 0.5100] | 134.358
 519 | [0.1500 1.2100 -0.4000 0.6100] | 158.254
 520 | [0.1500 1.2100 -0.4000 0.7100] | 189.748
 521 | [0.1500 1.2100 -0.3500 0.0100] |  73.324
 522 | [0.1500 1.2100 -0.3500 0.1100] |  75.301
 523 | [0.1500 1.2100 -0.3500 0.2100] |  80.748
 524 | [0.1500 1.2100 -0.3500 0.3100] |  90.135
 525 | [0.1500 1.2100 -0.3500 0.4100] | 104.219
 526 | [0.1500 1.2100 -0.3500 0.5100] | 124.049
 527 | [0.1500 1.2100 -0.3500 0.6100] | 150.990
 528 | [0.1500 1.2100 -0.3500 0.7100] | 186.685
 529 | [0.1500 1.2100 -0.3000 0.0100] |  58.492
 530 | [0.1500 1.2100 -0.3000 0.1100] |  60.632
 531 | [0.1500 1.2100 -0.3000 0.2100] |  66.573
 532 | [0.1500 1.2100 -0.3000 0.3100] |  76.926
 533 | [0.1500 1.2100 -0.3000 0.4100] |  92.641
 534 | [0.1500 1.2100 -0.3000 0.5100] | 114.991
 535 | [0.1500 1.2100 -0.3000 0.6100] | 145.549
 536 | [0.1500 1.2100 -0.3000 0.7100] | 

In [10]:
mybounds=((0,0.21),(0,5), (-0.4,0.3), (0, 0.801))

In [11]:
minimum=spo.minimize(Merton_error_function,opt_brute, method="L-BFGS-B",bounds=mybounds)

 641 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 642 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 643 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 644 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 645 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 646 | [0.0000 2.0030 -0.4000 0.0000] | 121.885
 647 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 648 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 649 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 650 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 651 | [0.1500 0.4100 -0.1500 0.0100] |   4.500
 652 | [0.0750 1.2065 -0.2750 0.0050] |  38.761
 653 | [0.0750 1.2065 -0.2750 0.0050] |  38.761
 654 | [0.0750 1.2065 -0.2750 0.0050] |  38.761
 655 | [0.0750 1.2065 -0.2750 0.0050] |  38.761
 656 | [0.0750 1.2065 -0.2750 0.0050] |  38.761
 657 | [0.1464 0.4485 -0.1560 0.0098] |   4.322
 658 | [0.1464 0.4485 -0.1560 0.0098] |   4.322
 659 | [0.1464 0.4485 -0.1560 0.0098] |   4.322
 660 | [0.1464 0.4485 -0.1560 0.0098] |   4.322
 661 | [0.1464 0.4485 -0.1560 0.0098] | 

 813 | [0.1253 0.1917 -0.4000 0.0540] |   2.812
 814 | [0.1253 0.1917 -0.4000 0.0540] |   2.812
 815 | [0.1253 0.1917 -0.4000 0.0540] |   2.812
 816 | [0.1253 0.1917 -0.4000 0.0540] |   2.812
 817 | [0.1274 0.1867 -0.4000 0.0544] |   2.804
 818 | [0.1274 0.1867 -0.4000 0.0544] |   2.804
 819 | [0.1274 0.1867 -0.4000 0.0544] |   2.804
 820 | [0.1274 0.1867 -0.4000 0.0544] |   2.804
 821 | [0.1274 0.1867 -0.4000 0.0544] |   2.804
 822 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 823 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 824 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 825 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 826 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 827 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 828 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 829 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 830 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 831 | [0.1271 0.1878 -0.4000 0.0544] |   2.804
 832 | [0.1271 0.1878 -0.4000 0.0546] |   2.804
 833 | [0.1271 0.1878 -0.4000 0.0546] | 

In [14]:
(0.1003, 0.4778, -0.2328, 0.1741) 
minimum

      fun: array([2.7393])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.0000, -0.0000, 2.0776, -0.0000])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 280
      nit: 42
     njev: 56
   status: 0
  success: True
        x: array([0.1221, 0.1911, -0.4000, 0.1901])

In [12]:
opt = lattice(Merton_error_function, 4, nbins=200, xtol=0.000001, ftol=0.000001,bounds=mybounds, maxiter=None, maxfun=None)

 917 | [0.0210 1.2500 -0.3125 0.0801] |  51.321
 918 | [0.0315 1.2500 -0.3125 0.0801] |  51.679
 919 | [0.0210 1.5000 -0.3125 0.0801] |  63.090
 920 | [0.0210 1.2500 -0.2775 0.0801] |  38.991
 921 | [0.0210 1.2500 -0.3125 0.1202] |  52.894
 922 | [0.0263 1.0000 -0.2950 0.1001] |  34.046
 923 | [0.0289 0.7500 -0.2863 0.1101] |  18.127
 924 | [0.0302 1.0000 -0.2819 0.0551] |  28.913
 925 | [0.0190 0.8750 -0.2666 0.0826] |  18.694
 926 | [0.0285 0.6875 -0.2436 0.0839] |   7.478
 927 | [0.0323 0.4062 -0.2091 0.0857] |  21.064
 928 | [0.0323 0.4062 -0.2616 0.0857] |  14.531
 929 | [0.0242 0.3594 -0.2471 0.1261] |  19.066
 930 | [0.0257 0.5195 -0.2558 0.1083] |   9.995
 931 | [0.0387 0.3066 -0.2571 0.1114] |  19.965
 932 | [0.0239 0.7329 -0.2642 0.0898] |  11.764
 933 | [0.0264 0.4231 -0.2264 0.0737] |  18.734
 934 | [0.0283 0.6683 -0.2713 0.1010] |  10.637
 935 | [0.0209 0.8979 -0.2558 0.1058] |  17.340
 936 | [0.0295 0.5292 -0.2602 0.0907] |   9.247
 937 | [0.0320 0.4693 -0.2512 0.1022] | 

1090 | [0.1174 0.2664 -0.3242 0.1592] |   2.931
1091 | [0.1177 0.2451 -0.3370 0.1611] |   2.967
1092 | [0.1201 0.2582 -0.3257 0.1575] |   2.929
1093 | [0.1207 0.2475 -0.3337 0.1599] |   2.906
1094 | [0.1217 0.2415 -0.3379 0.1611] |   2.901
1095 | [0.1241 0.2282 -0.3417 0.1592] |   2.918
1096 | [0.1234 0.2214 -0.3482 0.1617] |   2.917
1097 | [0.1229 0.2351 -0.3413 0.1646] |   2.903
1098 | [0.1198 0.2471 -0.3373 0.1651] |   2.893
1099 | [0.1176 0.2566 -0.3351 0.1680] |   2.909
1100 | [0.1186 0.2668 -0.3252 0.1643] |   2.960
1101 | [0.1222 0.2328 -0.3425 0.1623] |   2.894
1102 | [0.1237 0.2256 -0.3490 0.1653] |   2.884
1103 | [0.1257 0.2121 -0.3583 0.1674] |   2.884
1104 | [0.1218 0.2317 -0.3467 0.1633] |   2.873
1105 | [0.1212 0.2299 -0.3493 0.1627] |   2.866
1106 | [0.1227 0.2195 -0.3558 0.1677] |   2.860
1107 | [0.1232 0.2084 -0.3648 0.1710] |   2.867
1108 | [0.1225 0.2216 -0.3579 0.1691] |   2.846
1109 | [0.1226 0.2159 -0.3656 0.1725] |   2.829
1110 | [0.1263 0.1916 -0.3772 0.1700] | 

1261 | [0.0635 0.7609 -0.2027 0.0790] |   5.082
1262 | [0.0594 0.7429 -0.2075 0.1521] |   4.502
1263 | [0.0639 0.8304 -0.1935 0.1153] |   4.371
1264 | [0.0667 0.8993 -0.1829 0.1010] |   4.312
1265 | [0.0652 0.8606 -0.1906 0.1247] |   4.328
1266 | [0.0710 0.9697 -0.1671 0.0911] |   4.551
1267 | [0.0678 0.9044 -0.1794 0.1049] |   4.306
1268 | [0.0628 0.8502 -0.1939 0.1443] |   4.634
1269 | [0.0657 0.8526 -0.1882 0.1088] |   4.358
1270 | [0.0733 1.0155 -0.1631 0.0676] |   4.635
1271 | [0.0628 0.8110 -0.1964 0.1310] |   4.352
1272 | [0.0655 0.8851 -0.1865 0.1219] |   4.279
1273 | [0.0654 0.9013 -0.1856 0.1285] |   4.393
1274 | [0.0697 0.9636 -0.1733 0.0953] |   4.229
1275 | [0.0732 1.0399 -0.1618 0.0775] |   4.378
1276 | [0.0697 0.9657 -0.1704 0.0868] |   4.487
1277 | [0.0663 0.8868 -0.1856 0.1152] |   4.249
1278 | [0.0679 0.9206 -0.1795 0.1177] |   4.178
1279 | [0.0686 0.9313 -0.1778 0.1261] |   4.195
1280 | [0.0670 0.9237 -0.1830 0.1202] |   4.365
1281 | [0.0676 0.9092 -0.1803 0.1087] | 

1434 | [0.1078 0.8412 -0.1482 0.1095] |   3.712
1435 | [0.1077 0.8443 -0.1477 0.1117] |   3.713
1436 | [0.1094 0.8200 -0.1494 0.1052] |   3.711
1437 | [0.1107 0.8044 -0.1501 0.1031] |   3.712
1438 | [0.1069 0.8477 -0.1494 0.1050] |   3.711
1439 | [0.1056 0.8605 -0.1499 0.1035] |   3.712
1440 | [0.1062 0.8462 -0.1496 0.1086] |   3.709
1441 | [0.1047 0.8534 -0.1501 0.1105] |   3.709
1442 | [0.1088 0.8305 -0.1488 0.1073] |   3.709
1443 | [0.1078 0.8310 -0.1504 0.1036] |   3.708
1444 | [0.1078 0.8260 -0.1516 0.1006] |   3.710
1445 | [0.1054 0.8578 -0.1497 0.1071] |   3.708
1446 | [0.1072 0.8350 -0.1499 0.1082] |   3.705
1447 | [0.1073 0.8287 -0.1502 0.1098] |   3.704
1448 | [0.1046 0.8513 -0.1511 0.1072] |   3.709
1449 | [0.1077 0.8357 -0.1494 0.1073] |   3.707
1450 | [0.1079 0.8304 -0.1503 0.1053] |   3.704
1451 | [0.1100 0.8051 -0.1504 0.1059] |   3.707
1452 | [0.1087 0.8189 -0.1497 0.1106] |   3.704
1453 | [0.1093 0.8058 -0.1509 0.1086] |   3.701
1454 | [0.1101 0.7909 -0.1516 0.1092] | 

1606 | [0.1153 0.2928 -0.3041 0.1607] |   3.001
1607 | [0.1169 0.2815 -0.3082 0.1579] |   3.001
1608 | [0.1144 0.2847 -0.3100 0.1688] |   3.000
1609 | [0.1148 0.2705 -0.3178 0.1738] |   3.025
1610 | [0.1147 0.2848 -0.3123 0.1613] |   2.983
1611 | [0.1151 0.2754 -0.3194 0.1583] |   2.973
1612 | [0.1147 0.2644 -0.3224 0.1677] |   3.066
1613 | [0.1143 0.2969 -0.3045 0.1619] |   2.996
1614 | [0.1165 0.2834 -0.3106 0.1573] |   2.974
1615 | [0.1149 0.2774 -0.3181 0.1625] |   2.967
1616 | [0.1147 0.2697 -0.3252 0.1634] |   2.958
1617 | [0.1158 0.2780 -0.3198 0.1516] |   2.959
1618 | [0.1167 0.2563 -0.3329 0.1533] |   2.946
1619 | [0.1178 0.2360 -0.3472 0.1490] |   2.988
1620 | [0.1146 0.2563 -0.3380 0.1560] |   2.965
1621 | [0.1158 0.2548 -0.3386 0.1539] |   2.931
1622 | [0.1162 0.2445 -0.3482 0.1517] |   2.921
1623 | [0.1170 0.2679 -0.3250 0.1540] |   2.936
1624 | [0.1165 0.2413 -0.3459 0.1596] |   2.955
1625 | [0.1185 0.2353 -0.3509 0.1459] |   2.908
1626 | [0.1205 0.2181 -0.3637 0.1372] | 

...

34803 | [0.1990 0.0005 -0.1865 0.0181] |   9.304
34804 | [0.1990 0.0005 -0.1866 0.0183] |   9.302
34805 | [0.1989 0.0007 -0.1873 0.0193] |   9.294
34806 | [0.1989 0.0005 -0.1871 0.0189] |   9.296
34807 | [0.1989 0.0007 -0.1875 0.0195] |   9.292
34808 | [0.1989 0.0009 -0.1884 0.0207] |   9.282
34809 | [0.1988 0.0011 -0.1887 0.0214] |   9.277
34810 | [0.1987 0.0015 -0.1902 0.0235] |   9.260
34811 | [0.1987 0.0014 -0.1900 0.0231] |   9.262
34812 | [0.1987 0.0017 -0.1908 0.0244] |   9.254
34813 | [0.1986 0.0023 -0.1926 0.0271] |   9.233
34814 | [0.1985 0.0024 -0.1933 0.0279] |   9.224
34815 | [0.1983 0.0032 -0.1963 0.0322] |   9.190
34816 | [0.1983 0.0033 -0.1962 0.0322] |   9.191
34817 | [0.1982 0.0038 -0.1977 0.0344] |   9.176
34818 | [0.1980 0.0050 -0.2015 0.0400] |   9.135
34819 | [0.1978 0.0054 -0.2031 0.0423] |   9.116
34820 | [0.1974 0.0073 -0.2096 0.0516] |   9.049
34821 | [0.1974 0.0071 -0.2092 0.0510] |   9.052
34822 | [0.1972 0.0080 -0.2121 0.0552] |   9.024
34823 | [0.1967 0.01

34973 | [0.2098 4.1312 0.1190 0.0981] | 109.207
34974 | [0.2094 3.9413 0.1273 0.0627] | 104.387
34975 | [0.2097 3.7691 0.1233 0.0563] |  99.367
34976 | [0.2089 4.0676 0.1351 0.0558] | 109.094
34977 | [0.2097 3.6664 0.1190 0.0863] |  99.696
34978 | [0.2087 3.5867 0.1338 0.0416] |  99.463
34979 | [0.2096 3.4099 0.1171 0.0767] |  93.279
34980 | [0.2099 3.0810 0.1081 0.0872] |  85.798
34981 | [0.2092 3.7293 0.1246 0.0744] | 101.213
34982 | [0.2098 3.3223 0.1175 0.0612] |  90.385
34983 | [0.2094 3.2132 0.1223 0.0369] |  88.403
34984 | [0.2092 3.4666 0.1258 0.0510] |  94.938
34985 | [0.2095 2.7724 0.1136 0.0618] |  79.468
34986 | [0.2094 2.2741 0.1087 0.0646] |  68.932
34987 | [0.2100 2.4788 0.1025 0.0740] |  71.914
34988 | [0.2094 2.2012 0.1034 0.0701] |  66.432
34989 | [0.2093 1.6407 0.0963 0.0745] |  53.705
34990 | [0.2099 1.5241 0.0855 0.1132] |  52.252
34991 | [0.2094 0.8778 0.0884 0.0760] |  35.334
34992 | [0.2089 0.6796 0.0870 0.0902] |  30.567
34993 | [0.2094 0.0870 0.0698 0.1123] | 

35143 | [0.1960 4.3957 -0.0254 0.0219] |  63.062
35144 | [0.1951 4.0418 -0.0198 0.0100] |  58.744
35145 | [0.1949 3.8127 -0.0138 0.0104] |  56.413
35146 | [0.1924 3.8304 -0.0289 0.0194] |  55.765
35147 | [0.1901 3.5398 -0.0335 0.0274] |  52.125
35148 | [0.1905 3.4184 -0.0273 0.0034] |  50.263
35149 | [0.1885 3.0317 -0.0573 0.0150] |  47.679
35150 | [0.1849 2.3837 -0.0830 0.0185] |  41.659
35151 | [0.1878 2.6291 -0.0267 0.0251] |  40.654
35152 | [0.1854 1.9696 -0.0139 0.0352] |  31.909
35153 | [0.1805 1.8431 -0.0651 0.0319] |  30.151
35154 | [0.1733 0.8583 -0.0907 0.0426] |  14.462
35155 | [0.1770 0.7753 -0.0740 0.0224] |  13.320
35156 | [0.1853 2.4576 -0.0464 0.0166] |  38.302
35157 | [0.1756 0.6466 -0.0295 0.0399] |  10.451
35158 | [0.1816 1.7600 -0.0492 0.0258] |  28.103
35159 | [0.1683 0.0505 -0.1078 0.0301] |   7.198
35160 | [0.1776 1.1714 -0.0623 0.0298] |  19.016
35161 | [0.1756 0.8770 -0.0689 0.0318] |  14.214
35162 | [0.1749 0.3164 -0.0494 0.0195] |   6.756
35163 | [0.1757 0.04

35314 | [0.2042 0.5063 0.0296 0.1018] |  21.069
35315 | [0.2054 0.0011 -0.0073 0.1105] |  11.011
35316 | [0.2050 0.2565 0.0196 0.1045] |  15.938
35317 | [0.2040 0.2595 -0.0092 0.1234] |  15.483
35318 | [0.2008 0.0694 -0.0628 0.1657] |  11.024
35319 | [0.2042 0.1776 -0.0026 0.1165] |  13.951
35320 | [0.2038 0.1689 -0.0161 0.1251] |  13.610
35321 | [0.2038 0.1269 -0.0145 0.1227] |  12.826
35322 | [0.2036 0.1172 -0.0211 0.1267] |  12.574
35323 | [0.2036 0.0950 -0.0211 0.1261] |  12.162
35324 | [0.2035 0.0875 -0.0244 0.1280] |  11.989
35325 | [0.1890 3.7500 0.2125 0.5607] | 460.718
35326 | [0.1995 3.7500 0.2125 0.5607] | 460.257
35327 | [0.1890 4.0000 0.2125 0.5607] | 479.255
35328 | [0.1890 3.7500 0.2475 0.5607] | 478.107
35329 | [0.1890 3.7500 0.2125 0.6008] | 503.252
35330 | [0.1942 3.8750 0.2300 0.5206] | 436.617
35331 | [0.1969 3.9375 0.2387 0.4806] | 404.983
35332 | [0.1982 3.5938 0.2431 0.5207] | 423.312
35333 | [0.2028 3.7656 0.2059 0.5006] | 395.868
35334 | [0.2097 3.7734 0.1852 0

35485 | [0.1828 3.2063 -0.0506 0.0232] |  46.220
35486 | [0.1768 2.9722 -0.0501 0.0229] |  40.624
35487 | [0.1742 2.8724 -0.0821 0.0171] |  43.112
35488 | [0.1726 2.7963 -0.0529 0.0480] |  37.878
35489 | [0.1645 2.4694 -0.0458 0.0703] |  30.839
35490 | [0.1593 2.3074 -0.0786 0.0584] |  29.942
35491 | [0.1441 1.7365 -0.0964 0.0782] |  19.644
35492 | [0.1477 1.8259 -0.0725 0.0503] |  17.324
35493 | [0.1305 1.1393 -0.0763 0.0534] |   4.570
35494 | [0.1338 1.2863 -0.0523 0.0953] |   6.021
35495 | [0.1096 0.3436 -0.0854 0.1257] |  17.479
35496 | [0.1470 1.7979 -0.0617 0.0792] |  16.735
35497 | [0.1164 0.5470 -0.0414 0.0986] |  14.582
35498 | [0.1542 2.0417 -0.0305 0.0376] |  18.545
35499 | [0.1208 0.7681 -0.0717 0.1037] |   8.595
35500 | [0.1037 0.0724 -0.0591 0.0963] |  24.606
35501 | [0.1362 1.3665 -0.0611 0.0835] |   7.403
35502 | [0.1443 1.7331 -0.0893 0.0694] |  17.965
35503 | [0.1233 0.8435 -0.0534 0.0913] |   8.371
35504 | [0.1411 1.5497 -0.0499 0.0581] |   9.425
35505 | [0.1259 0.96

In [15]:
opt

array([0.1219, 0.1916, -0.3999, 0.1895])

In [13]:
opt_sprarsity = sparsity(Merton_error_function, 4, npts=200, xtol=0.0001, ftol=0.0001,bounds=mybounds, maxiter=None, maxfun=None)

35508 | [0.1812 1.2162 0.2313 0.3806] | 129.179
35509 | [0.1917 1.2162 0.2313 0.3806] | 131.736
35510 | [0.1812 1.4662 0.2313 0.3806] | 149.718
35511 | [0.1812 1.2162 0.2663 0.3806] | 142.524
35512 | [0.1812 1.2162 0.2313 0.4206] | 143.734
35513 | [0.1865 0.9662 0.2488 0.4006] | 119.989
35514 | [0.1891 0.7162 0.2576 0.4106] |  99.347
35515 | [0.1904 0.9662 0.2619 0.3555] | 112.874
35516 | [0.1950 0.8412 0.2247 0.3831] |  97.855
35517 | [0.2019 0.6537 0.2040 0.3843] |  77.093
35518 | [0.1896 0.5599 0.2461 0.3850] |  73.785
35519 | [0.1885 0.2318 0.2534 0.3871] |  35.778
35520 | [0.2037 0.0677 0.2571 0.3882] |  19.013
35521 | [0.1931 0.6918 0.2525 0.3741] |  88.344
35522 | [0.2045 0.1063 0.2260 0.3563] |  21.889
35523 | [0.1964 0.4783 0.2438 0.3765] |  64.806
35524 | [0.2001 0.4374 0.2245 0.3807] |  58.379
35525 | [0.1978 0.3446 0.2420 0.3773] |  49.845
35526 | [0.1994 0.3125 0.2346 0.3790] |  45.697
35527 | [0.2003 0.0146 0.2435 0.3780] |  11.142
35528 | [0.1993 0.2088 0.2398 0.3782] | 

35846 | [0.1267 0.1841 -0.4000 0.1401] |   2.767
35847 | [0.1263 0.1848 -0.4000 0.1345] |   2.767
35848 | [0.1259 0.1852 -0.4000 0.1347] |   2.767
35849 | [0.1263 0.1833 -0.4000 0.1528] |   2.765
35850 | [0.1261 0.1824 -0.4000 0.1681] |   2.765
35851 | [0.1260 0.1852 -0.3999 0.1442] |   2.761
35852 | [0.1256 0.1860 -0.3999 0.1473] |   2.758
35853 | [0.1255 0.1854 -0.3999 0.1458] |   2.762
35854 | [0.1250 0.1856 -0.3999 0.1533] |   2.760
35855 | [0.1253 0.1850 -0.3999 0.1649] |   2.755
35856 | [0.1249 0.1848 -0.3998 0.1800] |   2.753
35857 | [0.1242 0.1876 -0.3998 0.1605] |   2.751
35858 | [0.1232 0.1898 -0.3998 0.1643] |   2.749
35859 | [0.1239 0.1877 -0.3998 0.1767] |   2.745
35860 | [0.1231 0.1889 -0.3998 0.1922] |   2.743
35861 | [0.1234 0.1891 -0.3997 0.1886] |   2.745
35862 | [0.1217 0.1902 -0.3996 0.2152] |   2.751
35863 | [0.1207 0.1942 -0.3996 0.2002] |   2.744
35864 | [0.1235 0.1907 -0.3998 0.1574] |   2.748
35865 | [0.1222 0.1917 -0.3996 0.2049] |   2.755
35866 | [0.1229 0.19

36015 | [0.0000 0.2500 0.3000 0.0000] |  17.643
36016 | [0.0000 0.0000 0.3000 0.0000] | 108.076
36017 | [0.0000 0.0000 0.3000 0.0401] | 107.287
36018 | [0.0053 0.1250 0.3000 0.0200] |  28.026
36019 | [0.0079 0.1875 0.3000 0.0300] |  21.565
36020 | [0.0030 0.0703 0.3000 0.0263] |  34.953
36021 | [0.0073 0.0791 0.3000 0.0095] |  33.665
36022 | [0.0072 0.2505 0.3000 0.0035] |  17.612
36023 | [0.0029 0.3274 0.3000 0.0172] |  16.899
36024 | [0.0008 0.4515 0.3000 0.0211] |  21.238
36025 | [0.0038 0.3827 0.3000 0.0054] |  18.052
36026 | [0.0057 0.2451 0.3000 0.0183] |  17.827
36027 | [0.0042 0.1538 0.3000 0.0141] |  24.865
36028 | [0.0039 0.3255 0.3000 0.0076] |  16.873
36029 | [0.0046 0.2667 0.3000 0.0127] |  17.152
36030 | [0.0093 0.3350 0.3000 0.0205] |  16.889
36031 | [0.0031 0.3768 0.3000 0.0255] |  17.904
36032 | [0.0062 0.2821 0.3000 0.0090] |  16.901
36033 | [0.0066 0.3683 0.3000 0.0145] |  17.592
36034 | [0.0051 0.2921 0.3000 0.0131] |  16.851
36035 | [0.0044 0.3579 0.3000 0.0202] | 

36186 | [0.0080 0.3435 -0.3598 0.0147] |  13.388
36187 | [0.0105 0.3494 -0.3629 0.0216] |  13.053
36188 | [0.0052 0.2930 -0.3676 0.0329] |  15.753
36189 | [0.0084 0.2835 -0.3388 0.0209] |  18.101
36190 | [0.0064 0.3942 -0.3699 0.0211] |  13.270
36191 | [0.0096 0.4722 -0.3526 0.0032] |  14.598
36192 | [0.0085 0.4274 -0.3564 0.0106] |  13.134
36193 | [0.0071 0.4636 -0.3588 0.0194] |  14.984
36194 | [0.0077 0.3736 -0.3595 0.0159] |  12.721
36195 | [0.0093 0.4026 -0.3434 0.0104] |  12.177
36196 | [0.0107 0.4069 -0.3302 0.0051] |  11.970
36197 | [0.0083 0.3592 -0.3439 0.0181] |  13.172
36198 | [0.0084 0.4104 -0.3533 0.0125] |  12.573
36199 | [0.0053 0.4676 -0.3325 0.0025] |  12.380
36200 | [0.0114 0.3859 -0.3398 0.0031] |  12.315
36201 | [0.0083 0.3956 -0.3492 0.0108] |  12.370
36202 | [0.0087 0.4122 -0.3456 0.0089] |  12.250
36203 | [0.0142 0.3327 -0.3499 0.0114] |  13.883
36204 | [0.0075 0.4339 -0.3369 0.0048] |  12.058
36205 | [0.0108 0.4238 -0.3270 0.0001] |  11.703
36206 | [0.0074 0.45

36355 | [0.0034 1.0582 -0.1885 0.0095] |   6.630
36356 | [0.0012 1.1385 -0.1830 0.0121] |   6.378
36357 | [0.0040 1.0558 -0.1923 0.0152] |   6.586
36358 | [0.0007 1.1420 -0.1860 0.0133] |   6.419
36359 | [0.0026 1.0887 -0.1890 0.0131] |   6.481
36360 | [0.0014 1.1851 -0.1826 0.0027] |   6.336
36361 | [0.0019 1.1198 -0.1866 0.0102] |   6.414
36362 | [0.0011 1.1437 -0.1851 0.0099] |   6.389
36363 | [0.0006 1.1827 -0.1813 0.0027] |   6.283
36364 | [0.0011 1.1523 -0.1840 0.0073] |   6.369
36365 | [0.0008 1.1916 -0.1805 0.0112] |   6.247
36366 | [0.0004 1.2185 -0.1783 0.0162] |   6.155
36367 | [0.0006 1.2308 -0.1801 0.0024] |   6.282
36368 | [0.0004 1.2562 -0.1771 0.0047] |   6.153
36369 | [0.0001 1.3081 -0.1736 0.0034] |   6.108
36370 | [0.0009 1.2101 -0.1804 0.0044] |   6.249
36371 | [0.0004 1.3010 -0.1749 0.0105] |   6.184
36372 | [0.0003 1.2880 -0.1735 0.0149] |   6.037
36373 | [0.0002 1.3166 -0.1703 0.0211] |   5.972
36374 | [0.0006 1.2481 -0.1774 0.0086] |   6.147
36375 | [0.0003 1.24

...

57565 | [0.2100 4.6939 0.0844 0.3050] | 199.595
57566 | [0.2100 4.4452 0.0654 0.4100] | 274.729
57567 | [0.2100 4.5405 0.0835 0.3089] | 197.555
57568 | [0.2100 4.5538 0.1008 0.2451] | 163.603
57569 | [0.2100 4.6634 0.0849 0.2531] | 163.733
57570 | [0.2100 4.5261 0.1071 0.2059] | 145.545
57571 | [0.2100 4.8586 0.1304 0.1177] | 130.847
57572 | [0.2100 4.6071 0.1271 0.1059] | 122.313
57573 | [0.2100 4.5637 0.1485 0.0064] | 120.968
57574 | [0.2100 4.5877 0.1585 0.0345] | 128.125
57575 | [0.2100 4.5939 0.1185 0.1681] | 136.483
57576 | [0.2100 4.5885 0.1230 0.1438] | 130.234
57577 | [0.2100 4.6218 0.1293 0.1219] | 127.582
57578 | [0.2100 4.3223 0.1493 0.0356] | 118.957
57579 | [0.2100 4.5562 0.1347 0.0967] | 123.380
57580 | [0.2100 4.4443 0.1224 0.0958] | 115.280
57581 | [0.2100 4.3726 0.1043 0.1264] | 112.904
57582 | [0.2100 4.2856 0.1391 0.0107] | 111.965
57583 | [0.2100 4.4711 0.1350 0.0707] | 117.632
57584 | [0.2100 4.1621 0.1153 0.1153] | 111.754
57585 | [0.2100 4.3235 0.0976 0.1260] | 

57737 | [0.0149 0.2648 -0.3703 0.4968] |  11.295
57738 | [0.0166 0.2550 -0.3680 0.4891] |  11.553
57739 | [0.0141 0.2797 -0.3677 0.4988] |  11.258
57740 | [0.0128 0.2960 -0.3633 0.4976] |  11.546
57741 | [0.0153 0.2674 -0.3689 0.4959] |  11.233
57742 | [0.0150 0.2803 -0.3731 0.4959] |  11.131
57743 | [0.0153 0.2846 -0.3779 0.4955] |  11.191
57744 | [0.0144 0.2947 -0.3677 0.4954] |  11.408
57745 | [0.0148 0.2723 -0.3697 0.4964] |  11.179
57746 | [0.0152 0.2582 -0.3732 0.4999] |  11.436
57747 | [0.0146 0.2833 -0.3682 0.4952] |  11.171
57748 | [0.0157 0.2719 -0.3722 0.4929] |  11.046
57749 | [0.0165 0.2680 -0.3745 0.4899] |  10.986
57750 | [0.0152 0.2845 -0.3739 0.4928] |  11.102
57751 | [0.0158 0.2857 -0.3752 0.4904] |  11.042
57752 | [0.0167 0.2759 -0.3802 0.4893] |  10.874
57753 | [0.0177 0.2722 -0.3862 0.4863] |  10.754
57754 | [0.0177 0.2749 -0.3818 0.4838] |  10.723
57755 | [0.0190 0.2722 -0.3861 0.4778] |  10.558
57756 | [0.0194 0.2646 -0.3872 0.4795] |  10.625
57757 | [0.0205 0.25

57906 | [0.0074 0.8769 0.1809 0.0136] |  11.692
57907 | [0.0078 0.9060 0.1823 0.0123] |  11.791
57908 | [0.0020 0.9743 0.1627 0.0138] |  11.724
57909 | [0.0058 0.9864 0.1535 0.0119] |  12.704
57910 | [0.0053 0.8707 0.1793 0.0150] |  11.701
57911 | [0.0061 0.8918 0.1795 0.0110] |  11.642
57912 | [0.0067 0.8837 0.1853 0.0083] |  11.924
57913 | [0.0029 0.9290 0.1798 0.0145] |  11.723
57914 | [0.0089 0.8100 0.1970 0.0132] |  12.584
57915 | [0.0037 0.9332 0.1713 0.0137] |  11.533
57916 | [0.0084 0.8574 0.1757 0.0121] |  11.846
57917 | [0.0043 0.9111 0.1787 0.0139] |  11.631
57918 | [0.0054 0.9358 0.1759 0.0111] |  11.528
57919 | [0.0055 0.9683 0.1743 0.0092] |  11.482
57920 | [0.0024 0.9753 0.1709 0.0103] |  11.412
57921 | [0.0018 1.0021 0.1682 0.0125] |  11.347
57922 | [0.0024 1.0284 0.1636 0.0089] |  11.304
57923 | [0.0015 1.0870 0.1560 0.0064] |  11.325
57924 | [0.0024 1.0538 0.1672 0.0068] |  11.248
57925 | [0.0017 1.1141 0.1652 0.0034] |  11.247
57926 | [0.0038 0.9991 0.1706 0.0090] | 

58079 | [0.1665 2.3838 0.0589 0.0184] |  35.884
58080 | [0.1773 2.5281 0.0350 0.0746] |  41.408
58081 | [0.1630 2.7185 0.0087 0.0137] |  29.872
58082 | [0.1728 2.5493 0.0301 0.0513] |  36.933
58083 | [0.1658 2.4349 -0.0110 0.0770] |  30.471
58084 | [0.1609 2.4822 -0.0106 0.0417] |  26.354
58085 | [0.1550 2.4487 -0.0310 0.0369] |  23.221
58086 | [0.1615 2.6802 -0.0843 0.0839] |  38.832
58087 | [0.1653 2.4579 0.0231 0.0347] |  30.372
58088 | [0.1522 2.5042 0.0125 0.0042] |  22.617
58089 | [0.1623 2.4836 -0.0038 0.0497] |  27.654
58090 | [0.1510 2.6196 -0.0299 0.0175] |  22.403
58091 | [0.1439 2.7005 -0.0564 0.0089] |  22.371
58092 | [0.1437 2.3500 -0.0481 0.0362] |  18.197
58093 | [0.1341 2.1657 -0.0764 0.0474] |  16.473
58094 | [0.1543 2.4692 -0.0208 0.0370] |  22.769
58095 | [0.1373 2.4711 -0.0396 0.0119] |  15.052
58096 | [0.1481 2.4648 -0.0304 0.0276] |  19.737
58097 | [0.1294 2.3968 -0.1139 0.0437] |  26.707
58098 | [0.1465 2.4774 -0.0191 0.0141] |  18.444
58099 | [0.1391 2.0890 -0.

58249 | [0.1324 1.4087 -0.0715 0.1428] |  11.434
58250 | [0.1165 1.0143 -0.0546 0.1651] |   6.494
58251 | [0.1117 0.8362 -0.0924 0.1545] |   6.775
58252 | [0.1090 0.8559 -0.0807 0.1361] |   8.903
58253 | [0.1199 1.0674 -0.0676 0.1630] |   5.750
58254 | [0.1223 1.0907 -0.0900 0.1358] |   5.154
58255 | [0.1268 1.1572 -0.1058 0.1192] |   7.660
58256 | [0.1295 1.3395 -0.0549 0.1411] |   8.238
58257 | [0.1161 0.9621 -0.0830 0.1512] |   5.339
58258 | [0.1244 1.1353 -0.1069 0.1235] |   6.978
58259 | [0.1185 1.0446 -0.0677 0.1547] |   5.491
58260 | [0.1203 1.0709 -0.0940 0.1215] |   4.405
58261 | [0.1205 1.0726 -0.1072 0.1008] |   4.456
58262 | [0.1226 1.1068 -0.1070 0.1132] |   5.616
58263 | [0.1195 1.0601 -0.0775 0.1444] |   4.942
58264 | [0.1157 0.9128 -0.0899 0.1491] |   5.379
58265 | [0.1176 0.9793 -0.0880 0.1437] |   4.830
58266 | [0.1238 1.1385 -0.0917 0.1215] |   5.344
58267 | [0.1180 1.0062 -0.0852 0.1438] |   4.822
58268 | [0.1154 0.9675 -0.0823 0.1408] |   5.569
58269 | [0.1206 1.05

In [16]:
opt_sprarsity

array([0.1221, 0.1908, -0.4000, 0.1902])

In [20]:
Merton_error_function(opt)

58329 | [0.1219 0.1916 -0.3999 0.1895] |   2.740


array([2.7395])